## Task

Файл reviews.xlsx содержит датасет с рецензиями к фильмам с сайта **kinopoisk.ru**.

```
movie_link - ссылка на фильм
author_id – ссылка на страницу автора отзыва
author_name – имя автора
review_subtitle – заголовок отзыва
review_text – текст отзыва
target – оценка пользователя на фильм good / bad / neutral.
```

1. Необходимо построить модель по определению target 
2. Решении должно быть реализовано в Google Colab. Комментарии приветствуются.
3. Модель должна обучаться на 80% случайно выбранных записей, а финальная оценка точности должна быть проведена на оставшихся 20%. 
4. Оценка точности должна содержать общий показатель accuracy и отдельно по каждому классу precission, recall и f1-score.

## Import Library

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk
%matplotlib inline

## Import Dataset

In [3]:
df = pd.read_excel('reviews.xlsx')
df.head()

,target,movie_link,author_id,author_name,review_subtitle,review_text
0,good,/film/178563/,/user/311869/,sochi2014,NaN,Настоящее кино. Наше кино. Прекрасная работа с...
1,good,/film/178563/,/user/1591566/,Альберт Попов,Предтеча многого,"Мало того, что сериал «Империя под ударом» (12..."
2,good,/film/178563/,/user/331395/,LennoxL,Срез жизни общества начала 20 го века,Период от реформ Александра Освободителя до пе...
3,good,/film/178563/,/user/4379604/,Ana_K,"Пусть мир погибнет, но восторжествует правосудие",Атмосферный сериал о Российской Империи начала...
4,neutral,/film/178563/,/user/178788/,Skept,Террор  Зло с непредсказуемыми последствиями,Самая большая беда современного общества это т...


In [4]:
df.shape

(80284, 6)

In [5]:
df.columns

Index(['target', 'movie_link', 'author_id', 'author_name', 'review_subtitle',
       'review_text'],
      dtype='object')

#### Ищем дыры в датасете

In [6]:
df.isnull().sum()

target                 0
movie_link             0
author_id              0
author_name            2
review_subtitle    20084
review_text            2
dtype: int64

#### 1. Убираем все рецензии без текста и без автора
#### 2. Заменяем каждый NaN на Пусто

In [7]:
df = df[df['review_text'].notnull() & df['author_name'].notnull()]
df["review_subtitle"] = df["review_subtitle"].replace([float('nan')] , ["Пусто"])
df.isnull().sum()

target             0
movie_link         0
author_id          0
author_name        0
review_subtitle    0
review_text        0
dtype: int64

In [14]:
len(df['review_text'])

80280

In [26]:
reviews = df['review_text'].to_numpy()
reviews[0]

'Настоящее кино. Наше кино. Прекрасная работа сценариста, режиссёра, операторов. Профессиональный монтаж, музыкальное сопровождение. Превосходная игра актёров.\n\r\nРабота с\xa0таким тонким материалом, как\xa0история, требует мастерства и\xa0подлинного таланта. Это\xa0не фантазия и\xa0нахлынувшее воображение, а\xa0факты, подлинная история, с\xa0которой нельзя не\xa0считаться. Прочувственно то\xa0неспокойное время. Фильм не\xa0отвергает прошлое, как\xa0делают в\xa0некоторых картин, не\xa0старается выкрасить в\xa0чёрные цвета действующих лиц. \n\r\nФильм всего лишь повествует о\xa0людях, об\xa0идеях, о\xa0любви. Повествует о\xa0стране, которая очень сильно изменилась, но\xa0по-прежнему существует. Хочется вериться, что\xa0никакие идеи и\xa0люди, которые борются за\xa0эти идеи, не\xa0пойдут по\xa0проторённой дороге. Уроки должны быть выучены, особенно уроки истории.'

#### Можно заметить, что пробел почему-то иногда записывается, как '\xa0' и эмодзи, как '\n\r\n'